In [1]:
import numpy as np
import pandas as pd
import fuzzywuzzy as fw
import fuzzymatcher as fm
import recordlinkage as rl
from pathlib import Path
import time

/Users/sanjay/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/sanjay/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
hospital_accounts = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_info.csv'
)
hospital_reimbursement = pd.read_csv(
    'https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv'
)

In [3]:
hospital_accounts.head()

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
0,10605,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private
1,24250,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary
2,10341,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary
4,39835,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary


In [4]:
hospital_reimbursement.head()

,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,519118,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,43,13289.09,5413.63,4490.93
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,73,22261.60,4922.18,4021.79
3,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
4,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53


In [5]:
hospital_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5339 entries, 0 to 5338
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Account_Num         5339 non-null   int64 
 1   Facility Name       5339 non-null   object
 2   Address             5339 non-null   object
 3   City                5339 non-null   object
 4   State               5339 non-null   object
 5   ZIP Code            5339 non-null   int64 
 6   County Name         5339 non-null   object
 7   Phone Number        5339 non-null   object
 8   Hospital Type       5339 non-null   object
 9   Hospital Ownership  5339 non-null   object
dtypes: int64(2), object(8)
memory usage: 417.2+ KB


In [6]:
hospital_reimbursement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2697 entries, 0 to 2696
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Provider_Num               2697 non-null   int64  
 1   Provider Name              2697 non-null   object 
 2   Provider Street Address    2697 non-null   object 
 3   Provider City              2697 non-null   object 
 4   Provider State             2697 non-null   object 
 5   Provider Zip Code          2697 non-null   int64  
 6   Total Discharges           2697 non-null   int64  
 7   Average Covered Charges    2697 non-null   float64
 8   Average Total Payments     2697 non-null   float64
 9   Average Medicare Payments  2697 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 210.8+ KB


In [7]:
print(hospital_accounts.shape)
print(hospital_reimbursement.shape)

(5339, 10)
(2697, 10)


In [8]:
left_on = ["Facility Name", "Address", "City", "State"]
right_on = [
    "Provider Name", "Provider Street Address", "Provider City",
    "Provider State"]

In [9]:
left_on

['Facility Name', 'Address', 'City', 'State']

In [10]:
start = time.time()
matched_results = fm.fuzzy_left_join(hospital_accounts,
                                               hospital_reimbursement,
                                               left_on,
                                               right_on,
                                               left_id_col='Account_Num',
                                               right_id_col='Provider_Num')

In [11]:
cols = [
    "best_match_score", "Facility Name", "Provider Name", "Address", "Provider Street Address",
    "Provider City", "City", "Provider State", "State"
]

In [12]:
rearranged_best_matches=matched_results[cols].sort_values(by=['best_match_score'], ascending=False)
rearranged_best_matches.head(3)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
77823,3.090931,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,530 NEW BRUNSWICK AVE,530 NEW BRUNSWICK AVE,PERTH AMBOY,PERTH AMBOY,NJ,NJ
533239,2.799072,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ONE ROBERT WOOD JOHNSON PLACE,ONE ROBERT WOOD JOHNSON PLACE,NEW BRUNSWICK,NEW BRUNSWICK,NJ,NJ
78412,2.785132,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,1325 S CLIFF AVE POST OFFICE BOX 5045,1325 S CLIFF AVE POST OFFICE BOX 5045,SIOUX FALLS,SIOUX FALLS,SD,SD


In [13]:
rearranged_worst_matches=matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True)
rearranged_worst_matches.head(3)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
426752,-2.268231,CENTRO MEDICO WILMA N VAZQUEZ,BAPTIST MEDICAL CENTER EAST,CARR. 2 KM 39.5 ROAD NUMBER 2 BO ALGARROBO,400 TAYLOR ROAD,MONTGOMERY,VEGA BAJA,AL,PR
82938,-2.124071,DOCTOR CENTER HOSPITAL SAN FERNANDO DE LA CARO...,OVERLAKE HOSPITAL MEDICAL CENTER,EDIF JESUS T PINEIRO AVE FERNANDEZ JUNCOS BO P...,1035-116TH AVE NE,BELLEVUE,CAROLINA,WA,PR
42483,-2.106746,HOSPITAL ONCOLOGICO DR ISAAC GONZALEZ MARTINEZ,SCRIPPS MERCY HOSPITAL,BO. MONACILLOS CARR 22 CENTRO MEDICO DE PUERTO...,4077 5TH AVE,SAN DIEGO,SAN JUAN,CA,PR


In [14]:
matched_results[cols].query("best_match_score <= .85").sort_values(
    by=['best_match_score'], ascending=False).head(10)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
198991,0.846205,ASCENSION ALL SAINTS HOSPITAL,WHEATON FRANCISCAN HEALTHCARE ALL SAINTS,3801 SPRING ST,3801 SPRING ST,RACINE,RACINE,WI,WI
61868,0.827016,UNION HOSPITAL,UNION HOSPITAL,659 BOULEVARD,659 BOULEVARD,DOVER,DOVER,OH,OH
11058,0.826296,LAFAYETTE SURGICAL SPECIALTY HOSPITAL,HEART HOSPITAL OF LAFAYETTE,1101 KALISTE SALOOM RD,1105 KALISTE SALOOM ROAD,LAFAYETTE,LAFAYETTE,LA,LA
291363,0.816168,PRISMA HEALTH RICHLAND HOSPITAL,PALMETTO HEALTH RICHLAND,5 MEDICAL PARK,5 RICHLAND MEDICAL PARK,COLUMBIA,COLUMBIA,SC,SC
440394,0.815257,NORTHSIDE HOSPITAL GWINNETT,GWINNETT MEDICAL CENTER,1000 MEDICAL CENTER BOULEVARD,1000 MEDICAL CENTER BOULEVARD,LAWRENCEVILLE,LAWRENCEVILLE,GA,GA
122613,0.807292,MCLAREN THUMB REGION,HURON MEDICAL CENTER,1100 SOUTH VAN DYKE ROAD,1100 SOUTH VAN DYKE ROAD,BAD AXE,BAD AXE,MI,MI
561113,0.804354,ADVENTIST HEALTH PORTLAND,ADVENTIST MEDICAL CENTER,10123 SE MARKET STREET,10123 SE MARKET STREET,PORTLAND,PORTLAND,OR,OR
157301,0.803704,MERIT HEALTH WOMEN'S HOSPITAL,MERIT HEALTH RIVER OAKS,1026 RIVER OAKS DRIVE,1030 RIVER OAKS DRIVE,FLOWOOD,FLOWOOD,MS,MS
506360,0.800649,HIGHLAND HOSPITAL,HIGHLAND HOSPITAL,1411 EAST 31ST STREET,1411 E 31ST STREET,OAKLAND,OAKLAND,CA,CA
518787,0.792471,METHODIST HOSPITAL SOUTH,SOUTH TEXAS REGIONAL MEDICAL CENTER,1905 HWY 97 EAST,1905 HWY 97 EAST,JOURDANTON,JOURDANTON,TX,TX


In [15]:
end = time.time()
print("The time of execution of above program is :",
      (end-start), "sec")

The time of execution of above program is : 65.83530807495117 sec


In [16]:
hospital_accounts = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_info.csv', index_col='Account_Num'
)
hospital_reimbursement = pd.read_csv(
    'https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv', index_col='Provider_Num'
)

In [17]:
# Build the indexer
indexer = rl.Index()
# Can use full or block
indexer.full()
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
# Let's see how many matches we want to do
print(len(candidates))

14399283


In [18]:
indexer = rl.Index()
indexer.block(left_on='State', right_on='Provider State')
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
# Let's see how many matches we want to do
print(len(candidates))

475830


In [19]:
indexer = rl.Index()
# Use sortedneighbor as a good option if data is not clean
indexer.sortedneighbourhood(left_on='State', right_on='Provider State')

candidates = indexer.index(hospital_accounts, hospital_reimbursement)
# Let's see how many matches we want to do
print(len(candidates))

998860


In [20]:
start1 = time.time()
compare = rl.Compare()
compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
               'Provider Name',
               threshold=0.85,
               label='Hosp_Name',method='levenshtein')
compare.string('Address',
               'Provider Street Address',
               method='jarowinkler',
               threshold=0.85,
               label='Hosp_Address')
features = compare.compute(candidates, hospital_accounts,
                           hospital_reimbursement)

In [21]:
features

City  Hosp_Name  Hosp_Address
Account_Num Provider_Num                               
10605       537184           0        0.0           0.0
            803181           0        0.0           0.0
            450616           0        0.0           0.0
            854377           0        0.0           0.0
            560361           0        0.0           0.0
...                        ...        ...           ...
51586       469504           0        0.0           0.0
            898766           0        0.0           0.0
            664049           0        0.0           0.0
            552832           0        0.0           0.0
            857017           0        0.0           0.0

[998860 rows x 3 columns]

In [22]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      2290
2.0       461
1.0      8261
0.0    987848
Name: count, dtype: int64

In [23]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 1].reset_index()

In [24]:
potential_matches

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address
0,51216,268781,0,1.0,1.0
1,59519,603343,1,1.0,1.0
2,66832,297781,1,0.0,1.0
3,83753,151859,1,0.0,1.0
4,71617,123841,1,1.0,1.0
...,...,...,...,...,...
2746,18990,824535,1,1.0,1.0
2747,91079,445786,1,1.0,1.0
2748,26157,614010,1,0.0,1.0
2749,92281,501763,1,1.0,1.0


In [25]:
potential_matches['Score'] = potential_matches.loc[:, 'City':'Hosp_Address'].sum(axis=1)
potential_matches.reset_index()
potential_matches.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score
0,51216,268781,0,1.0,1.0,2.0
1,59519,603343,1,1.0,1.0,3.0
2,66832,297781,1,0.0,1.0,2.0
3,83753,151859,1,0.0,1.0,2.0
4,71617,123841,1,1.0,1.0,3.0


In [26]:
# Add some convenience columns for comparing data
hospital_accounts['Acct_Name_Lookup'] = hospital_accounts[[
    'Facility Name', 'Address', 'City', 'State'
]].apply(lambda x: '_'.join(x), axis=1)

In [27]:
hospital_accounts['Acct_Name_Lookup']

Account_Num
10605    SAGE MEMORIAL HOSPITAL_STATE ROUTE 264 SOUTH 1...
24250    WOODRIDGE BEHAVIORAL CENTER_600 NORTH 7TH STRE...
10341    DOUGLAS GARDENS HOSPITAL_5200 NE 2ND AVE_MIAMI_FL
81101    SUNCOAST BEHAVIORAL HEALTH CENTER_4480 51ST ST...
39835    TREASURE VALLEY HOSPITAL_8800 WEST EMERALD STR...
                               ...                        
92281    JEWISH HOSPITAL - SHELBYVILLE_727 HOSPITAL DRI...
65248    DAYTON CHILDREN'S HOSPITAL_ONE CHILDRENS PLAZA...
92377    NORTH TEXAS STATE HOSPITAL_6515 KEMP BLVD_WICH...
71562    GULF BREEZE HOSPITAL_1110 GULF BREEZE PKWY_GUL...
51586    I-70 COMMUNITY HOSPITAL_105 HOSPITAL DRIVE, BU...
Name: Acct_Name_Lookup, Length: 5339, dtype: object

In [28]:
hospital_reimbursement['Reimbursement_Name_Lookup'] = hospital_reimbursement[[
    'Provider Name', 'Provider Street Address', 'Provider City',
    'Provider State'
]].apply(lambda x: '_'.join(x), axis=1)

In [29]:
reimbursement_lookup = hospital_reimbursement[['Reimbursement_Name_Lookup']].reset_index()
account_lookup = hospital_accounts[['Acct_Name_Lookup']].reset_index()

In [30]:
account_lookup.head(2)

,Account_Num,Acct_Name_Lookup
0,10605,SAGE MEMORIAL HOSPITAL_STATE ROUTE 264 SOUTH 1...
1,24250,WOODRIDGE BEHAVIORAL CENTER_600 NORTH 7TH STRE...


In [31]:
reimbursement_lookup.head(2)

,Provider_Num,Reimbursement_Name_Lookup
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER_1108 ROSS CLA...
1,519118,MARSHALL MEDICAL CENTER SOUTH_2505 U S HIGHWAY...


In [32]:
account_merge = potential_matches.merge(account_lookup, how='left')
account_merge.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score,Acct_Name_Lookup
0,51216,268781,0,1.0,1.0,2.0,ST FRANCIS MEDICAL CENTER_2400 ST FRANCIS DRIV...
1,59519,603343,1,1.0,1.0,3.0,MARIA PARHAM MEDICAL CENTER_PO BOX 59_HENDERSO...
2,66832,297781,1,0.0,1.0,2.0,MEMORIAL CARE MILLER CHILDREN'S & WOMEN'S HOS...
3,83753,151859,1,0.0,1.0,2.0,ADVENTHEALTH LAKE WALES_410 S 11TH ST_LAKE WAL...
4,71617,123841,1,1.0,1.0,3.0,NEWARK BETH ISRAEL MEDICAL CENTER_201 LYONS AV...


In [33]:
final_merge = account_merge.merge(reimbursement_lookup, how='left')
final_merge.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
0,51216,268781,0,1.0,1.0,2.0,ST FRANCIS MEDICAL CENTER_2400 ST FRANCIS DRIV...,SAINT FRANCIS MEDICAL CENTER_211 ST FRANCIS DR...
1,59519,603343,1,1.0,1.0,3.0,MARIA PARHAM MEDICAL CENTER_PO BOX 59_HENDERSO...,MARIA PARHAM MEDICAL CENTER_PO BOX 59_HENDERSO...
2,66832,297781,1,0.0,1.0,2.0,MEMORIAL CARE MILLER CHILDREN'S & WOMEN'S HOS...,LONG BEACH MEMORIAL MEDICAL CENTER_2801 ATLANT...
3,83753,151859,1,0.0,1.0,2.0,ADVENTHEALTH LAKE WALES_410 S 11TH ST_LAKE WAL...,LAKE WALES MEDICAL CENTER_410 S 11TH ST_LAKE W...
4,71617,123841,1,1.0,1.0,3.0,NEWARK BETH ISRAEL MEDICAL CENTER_201 LYONS AV...,NEWARK BETH ISRAEL MEDICAL CENTER_201 LYONS AV...


In [34]:
cols = [
    'Account_Num', 'Provider_Num', 'Score', 'Acct_Name_Lookup',
    'Reimbursement_Name_Lookup'
]

In [35]:
final_merge[cols].sort_values(by=['Account_Num', 'Score'], ascending=False)

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
2376,94995,825914,3.0,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...
1862,94953,819181,3.0,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...
1644,94943,680596,3.0,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...
2235,94923,403151,3.0,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...
951,94887,752284,2.0,NEW YORK-PRESBYTERIAN BROOKLYN METHODIST HOSPI...,NEW YORK METHODIST HOSPITAL_506 SIXTH STREET_B...
...,...,...,...,...,...
427,10165,188247,3.0,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT
1683,10090,212069,3.0,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...
917,10043,140535,3.0,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...
1162,10020,210657,3.0,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...


In [36]:
end1 = time.time()
print("The time of execution of above program is :",
      (end1-start1), "sec")

The time of execution of above program is : 9.787724018096924 sec
